In [1]:
import shutil
from datetime import datetime as dt

# Reference: https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/
import pandas as pd
import glob
import os

import numpy as np
import pandas as pd

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dropout, Dense, RepeatVector
from tensorflow.keras.layers import LSTM

!pip install pyyaml h5py


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
cryptos_df = []
cryptos = ["BTC", "ADA", "BCH", "BNB", "DOGE", "ETH", "FTT", "LINK", "OKB", "SOL"]

for i in range(len(cryptos)):
    string = "./data/processed/" + cryptos[i]+"_imagepath.csv"
    crypto_df = pd.read_csv(string)
    
    df = crypto_df
    df["Date"] = crypto_df["Unnamed: 0"]
    df[['Day', 'Month', 'Year']] = df["Date"].str.split('/', n=2, expand=True)
    df['Day'] = [d if len(d) == 2 else '0'+d for d in df['Day']]
    df['Month'] = [m if len(m) == 2 else '0'+m for m in df['Month']]

    df['DateStr'] = df['Year'] + df['Month'] + df['Day']
    cryptos_df.append(df)

In [3]:
cryptos_df[-1]

,Unnamed: 0,RT_Short_Term_Candlesticks_Pathname,RT_Long_Term_Candlesticks_Pathname,Adj Close,Tmr_adjClose,Change in AdjClose,Percentage Change,Positive/Negative,Volume,STOCK_BTCM,...,Gemini_Exchange_Volume,Kraken_Exchange_Volume,KuCoin_Exchange_Volume,MEXC_Exchange_Volume,Crypto_LTC,Date,Day,Month,Year,DateStr
0,22/3/2023,NaN,NaN,21.480486,22.216778,0.736292,0.034277,1.0,851680842.0,2.410000,...,49293101.47,1.148069e+09,4.789541e+08,3.202973e+09,87.356888,22/3/2023,22,03,2023,20230322
1,23/3/2023,NaN,NaN,22.216778,20.678856,-1.537922,-0.069223,0.0,555730954.0,2.750000,...,44768048.96,1.004167e+09,4.815834e+08,1.438445e+09,93.701004,23/3/2023,23,03,2023,20230323
2,24/3/2023,NaN,NaN,20.678856,20.446646,-0.232210,-0.011229,0.0,547225198.0,2.470000,...,37313975.29,8.112928e+08,3.704522e+08,4.779163e+09,93.709908,24/3/2023,24,03,2023,20230324
3,25/3/2023,NaN,NaN,20.446646,20.975519,0.528873,0.025866,1.0,272949711.0,2.486667,...,16608231.19,2.876192e+08,2.743611e+08,7.819593e+08,91.581146,25/3/2023,25,03,2023,20230325
4,26/3/2023,NaN,NaN,20.975519,19.921675,-1.053844,-0.050242,0.0,226663968.0,2.503333,...,13077869.89,3.016346e+08,2.794530e+08,9.852968e+08,93.525192,26/3/2023,26,03,2023,20230326
5,27/3/2023,NaN,NaN,19.921675,20.470697,0.549022,0.027559,1.0,359607077.0,2.520000,...,33358329.65,7.342237e+08,3.612773e+08,3.870841e+08,89.246872,27/3/2023,27,03,2023,20230327
6,28/3/2023,NaN,NaN,20.470697,21.111467,0.640770,0.031302,1.0,347656506.0,2.310000,...,29134775.43,7.486706e+08,3.685084e+08,7.510869e+08,89.139374,28/3/2023,28,03,2023,20230328
7,29/3/2023,NaN,NaN,21.111467,20.551046,-0.560421,-0.026546,0.0,378726667.0,2.560000,...,31005442.49,8.836883e+08,4.503788e+08,8.485559e+08,91.341736,29/3/2023,29,03,2023,20230329
8,30/3/2023,NaN,NaN,20.551046,21.170496,0.619450,0.030142,1.0,472449267.0,2.400000,...,29044565.56,7.250529e+08,4.230337e+08,9.267878e+08,89.059891,30/3/2023,30,03,2023,20230330
9,31/3/2023,NaN,NaN,21.170496,20.975641,-0.194855,-0.009204,0.0,352864960.0,2.460000,...,23664343.22,7.283557e+08,3.936672e+08,4.812372e+08,89.684128,31/3/2023,31,03,2023,20230331


In [4]:
all_test_sets = []
for i in range(len(cryptos)):
    REQ_PATH = "./data/processed/images/" + cryptos[i]
    if not (os.path.isdir(REQ_PATH)):
        os.mkdir(REQ_PATH)
        os.mkdir(REQ_PATH + '/test_short')
        os.mkdir(REQ_PATH + '/test_long')
        os.mkdir(REQ_PATH + '/test_short/2')
        os.mkdir(REQ_PATH + '/test_long/2')
    path = "./data/processed/images/"

    all_files = glob.glob(os.path.join(path +"/*.png"))

    total_count = len(all_files)
    test_count = 0

    for path in all_files:
        if cryptos[i] not in path:
            continue
        filename = path[24:]
        new_path = REQ_PATH

        test_count += 1
        total_count += 1
        if filename[-9] == 's':
            new_path = os.path.join(new_path, 'test_short')
        else:
            new_path = os.path.join(new_path, 'test_long')
        new_path = os.path.join(new_path, '2')
        new_path = os.path.join(new_path, filename)
        shutil.copyfile(path, new_path)

    test_sets = []
    for period in ('long', 'short'):
        print(REQ_PATH + '/test_'+period)
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_set = test_datagen.flow_from_directory(
            REQ_PATH + '/test_'+period,
            target_size=(256, 256),
            color_mode='rgb',
            batch_size=1,
            class_mode='binary',
            shuffle=False)
        test_sets.append(test_set)
    all_test_sets.append(test_sets)

./data/processed/images/BTC/test_long
Found 1 images belonging to 1 classes.
./data/processed/images/BTC/test_short
Found 1 images belonging to 1 classes.
./data/processed/images/ADA/test_long
Found 1 images belonging to 1 classes.
./data/processed/images/ADA/test_short
Found 1 images belonging to 1 classes.
./data/processed/images/BCH/test_long
Found 1 images belonging to 1 classes.
./data/processed/images/BCH/test_short
Found 1 images belonging to 1 classes.
./data/processed/images/BNB/test_long
Found 1 images belonging to 1 classes.
./data/processed/images/BNB/test_short
Found 1 images belonging to 1 classes.
./data/processed/images/DOGE/test_long
Found 1 images belonging to 1 classes.
./data/processed/images/DOGE/test_short
Found 1 images belonging to 1 classes.
./data/processed/images/ETH/test_long
Found 1 images belonging to 1 classes.
./data/processed/images/ETH/test_short
Found 1 images belonging to 1 classes.
./data/processed/images/FTT/test_long
Found 1 images belonging to 1 

In [5]:
thresholds = pd.read_csv('./trained_parameters/threshold_CNN.csv')
import keras
from matplotlib import pyplot as plt
results = []
paths = ['./trained_parameters/CNN_trained_long', './trained_parameters/CNN_trained_short']
for i in range(len(cryptos)):
    test_sets = all_test_sets[i]
    for j in range(2):
        model_v = keras.models.load_model(paths[j])
        threshold = thresholds["Threshold"].iloc[j]
    
        filenames = test_sets[j].filenames
        nb_samples = len(filenames)
        predict = model_v.predict(test_sets[j], steps=nb_samples)
        pred_result = (predict >= threshold).astype(int).reshape(-1,1)
        result = np.concatenate((np.array(filenames).reshape(-1,1), pred_result), axis=1)
        date = [r[2:2+8] for r in result[:,0]]
        date = np.array(date)
        date = date.reshape(date.shape[0], 1)
        result = np.concatenate((result, date), axis=1)
        result = result[:, 1:]
        result = result[:, [1,0]]
        result = result[result[:,0].argsort()]
    
        df = pd.DataFrame({"Date" : result[:,0], "Prediction" : result[:,1]})
        results.append(df)

In [6]:
for i in range(len(cryptos_df)):
    cryptos_df[i].insert(1,'CNN_Prediction_long',np.nan)
    cryptos_df[i].insert(2,'CNN_Prediction_short',np.nan)
    
    cryptos_df[i]['CNN_Prediction_long'].iloc[-1] = results[i*2]['Prediction'].iloc[0]
    cryptos_df[i]['CNN_Prediction_short'].iloc[-1] = results[i*2+1]['Prediction'].iloc[0]
    
    cryptos_df[i].drop(columns=['RT_Short_Term_Candlesticks_Pathname', 'RT_Long_Term_Candlesticks_Pathname', 'Date', 'Day', 'Month', 'Year', 'DateStr'], inplace=True)

C:\Users\user\anaconda3\envs\COMP5214\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
for i in range(len(cryptos_df)): 
    o_string = "./data/processed/" + cryptos[i]+"_CNN_predicted.csv"
    cryptos_df[i].to_csv(o_string)